In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm

from matplotlib.dates import DateFormatter
dateformat = DateFormatter(fmt = '%H:%M:%S:%f')

In [2]:
def import_training_data(patient_num):
    
    # First, get the labels
    labels = pd.read_csv(f'../data/new_data/marked/labels/labels_{patient_num}.csv')
    sgb = labels['labels'].tolist()
        
    # Then the raw data
    readings = pd.read_csv(f'../data/new_data/eval_data/{patient_num} RAW 00.csv')
    readings['Time'] = pd.to_datetime(readings['Time'])
    readings['CapturedTime'] = readings['Time'].dt.time
    
    # Then remove the meal times
    events = pd.read_csv(f'../data/new_data/eval_data/{patient_num} EVT 00.csv')
    events = events[events['EventName'] == 'Meal']
    events['Time'] = pd.to_datetime(events['Time'])
    events['Duration'] = pd.to_timedelta(events['Duration'])
    events = events.assign(End = events['Time'] + events['Duration'])
    for idx, row in events.iterrows():
        readings = readings[(readings['Time'] < row['Time']) | (readings['Time'] > row['End'])]
    
#     # There is a section for patient 008 at the end which is all maxed outs
#     if patient_num == '008':
#         readings = readings.loc[:837833]
    
#     # There is a missing segment of time for 009, so we need to repair the index
#     if patient_num == '009':
#         readings.index = [x + 4 if x >= 276 else x for x in readings.index]
        
#     # There is a section for patient 010 at the end which is all maxed outs
#     if patient_num == '010':
#         readings = readings.loc[:731110]
    
    return sgb, readings

In [3]:
from ipywidgets import interact
import ipywidgets

In [4]:
# @interact(idx = ipywidgets.IntText(0),
#          shift = ipywidgets.IntSlider(value = 0, min = -20, max = 50))
def plot_event(idx, shift = 0):
    
    duration = 20
    
    idx = orig_index[idx]
    
    readings_sub = (
        readings
        .loc[idx - shift:idx - shift + duration - 1]
        [['Channel_' + str(i) for i in range(1,7)]]
        .values
    )

    fig, ax = plt.subplots(figsize = (12,6))

    times = readings.loc[idx - shift:idx - shift + duration - 1, 'Time']

    for i in range(6):
        if i == 0:
            color = 'blue'
        else:
            color = 'grey'
        
        plt.plot(times, readings_sub[:, i], label = 'Channel_' + str(i + 1), color = color)

    ax.xaxis.set_major_formatter(dateformat)

    plt.legend(bbox_to_anchor = (1, 0.5), loc = 'center left')
    
    xmin, xmax = plt.xlim()
    midpoint = (xmin + xmax) / 2
    
    plt.ylim(-500, 10500)
    
    ymin, ymax = plt.ylim()
    
    plt.vlines(x = midpoint, ymin = ymin, ymax = ymax, color = 'grey', linestyle = '--')
    
#     if filepath:
#         plt.tight_layout()
#         plt.savefig(filepath, 
#                     dpi = 150, 
#                     transparent = False, 
#                     facecolor = 'white')
        
#         plt.close();

In [5]:
button = ipywidgets.Button(description="Save")


def save_start(b):
    orig_idx = idx.value
    orig_idx = orig_index[orig_idx]
    
    shift_amt = shift.value
    
    new_idx = orig_idx - shift_amt
    
    new_indices.append((idx.value,new_idx))

button.on_click(save_start)

undo_button = ipywidgets.Button(description="Undo Last Save")

def remove_last(b): 
    global new_indices
    new_indices = new_indices[:-1]

undo_button.on_click(remove_last)



button_dc = ipywidgets.Button(description="Double Check")

def save_dc(b):
    orig_idx = idx.value
    orig_idx = orig_index[orig_idx]
    
    shift_amt = shift.value
    
    new_idx = orig_idx - shift_amt
    
    dc_indices.append((idx.value,new_idx))

button_dc.on_click(save_dc)


undo_dc = ipywidgets.Button(description="Undo Last DC")

def remove_last_dc(b): 
    global dc_indices
    dc_indices = dc_indices[:-1]

undo_dc.on_click(remove_last_dc)

In [22]:
patient_num = '89'

sgb, readings = import_training_data(patient_num)
readings.loc[readings['CapturedTime'].astype(str).isin(sgb), 'sgb'] = True
readings['sgb'] = readings['sgb'].fillna(False)
orig_index = readings.loc[readings['sgb']].index.tolist()

new_indices = []
dc_indices = []

In [24]:
idx = ipywidgets.IntText(0)
shift = ipywidgets.IntSlider(value = 0, min = -40, max = 50)

out = ipywidgets.interactive_output(plot_event, {'idx': idx, 'shift': shift})

In [25]:
len(orig_index)

644

In [26]:
ipywidgets.VBox([idx, 
                 shift, 
                 ipywidgets.HBox([button, undo_button]), 
                 ipywidgets.HBox([button_dc, undo_dc]), 
                 out])

In [32]:
new_indices[-3:]

[(537, 703094), (538, 705520), (539, 705555)]

In [11]:
dc_indices[-3:]

[]

In [34]:
centered_df = pd.DataFrame(
    {
        'patient': patient_num,
        'Time': readings.loc[[x[1] for x in new_indices]]['Time']
    }
)
filepath = f'../data/new_data/centered/centered_{patient_num}.csv'
centered_df.to_csv(filepath, index = False)


dc_df = pd.DataFrame(
    {
        'patient': patient_num,
        'Time': readings.loc[[x[1] for x in dc_indices]]['Time']
    }
)
filepath = f'../data/new_data/double_check/dc_{patient_num}.csv'
dc_df.to_csv(filepath, index = False)